### Dataframe example

In [1]:
import os
import pyspark
import re
import sys
from operator import add
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')
sc.uiWebUrl

21/10/19 21:09:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://39cf2f12b05e:4040'

In [3]:
# grades.csv should be:
# name,hw1,hw2,hw3
# john,26,29,30
# mary,27,25,35
# bill,19,22,25
# lee,25,25,25

gradeDF = spark.read.option("header", "true").csv("grades.csv")
gradeDF.show()

+----+---+---+---+
|name|hw1|hw2|hw3|
+----+---+---+---+
|john| 26| 29| 30|
|mary| 27| 25| 35|
|bill| 19| 22| 25|
| lee| 25| 25| 25|
+----+---+---+---+



In [4]:
gradeDF.toPandas().head()

,name,hw1,hw2,hw3
0,john,26,29,30
1,mary,27,25,35
2,bill,19,22,25
3,lee,25,25,25


In [5]:
gradeDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- hw1: string (nullable = true)
 |-- hw2: string (nullable = true)
 |-- hw3: string (nullable = true)



In [6]:
# majors.csv should be
# name,dept
# john,CSE
# mary,BIOLOGY
# bill,ECE
# lee,CIVIL

majorDF = spark.read.option("header","true").csv("majors.csv")
majorDF.show()

+----+-------+
|name|   dept|
+----+-------+
|john|    CSE|
|mary|BIOLOGY|
|bill|    ECE|
| lee|  CIVIL|
+----+-------+



### Join the grades and majors tables. Rename majorDF.name as majorDF.name2

In [7]:
result = gradeDF.join(majorDF, ['name'], how='inner')
result.show()

+----+---+---+---+-------+
|name|hw1|hw2|hw3|   dept|
+----+---+---+---+-------+
|john| 26| 29| 30|    CSE|
|mary| 27| 25| 35|BIOLOGY|
|bill| 19| 22| 25|    ECE|
| lee| 25| 25| 25|  CIVIL|
+----+---+---+---+-------+



### Calculate the total grade

In [8]:
hwtotalDF = result.selectExpr("name", "hw1+hw2+hw3 as hwgrade", "dept")
hwtotalDF.show()

+----+-------+-------+
|name|hwgrade|   dept|
+----+-------+-------+
|john|   85.0|    CSE|
|mary|   87.0|BIOLOGY|
|bill|   66.0|    ECE|
| lee|   75.0|  CIVIL|
+----+-------+-------+



### Find students with total grade greater than 80

In [9]:
hwtotalDF.filter(hwtotalDF.hwgrade > 80).show()

+----+-------+-------+
|name|hwgrade|   dept|
+----+-------+-------+
|john|   85.0|    CSE|
|mary|   87.0|BIOLOGY|
+----+-------+-------+



### Create Dataframe for college information

In [10]:
c = (('CSE','EGR'), ('ECE', 'EGR'), ('CIVIL', 'EGR'), ('BIOLOGY', 'NATSCI'))
crdd = sc.parallelize(c)
collegeDF = crdd.toDF(['deptname', 'collegename'])
collegeDF.show()

+--------+-----------+
|deptname|collegename|
+--------+-----------+
|     CSE|        EGR|
|     ECE|        EGR|
|   CIVIL|        EGR|
| BIOLOGY|     NATSCI|
+--------+-----------+



### Count the number of departments in each college

In [11]:
res = collegeDF.groupBy(collegeDF.collegename).agg({'deptname': 'count'})
res.show()

+-----------+---------------+
|collegename|count(deptname)|
+-----------+---------------+
|     NATSCI|              1|
|        EGR|              3|
+-----------+---------------+



### Compute Average grade of students in each college

In [12]:
studentsDF = hwtotalDF.join(collegeDF, hwtotalDF['dept'] == collegeDF['deptname'])
studentsDF.show()

+----+-------+-------+--------+-----------+
|name|hwgrade|   dept|deptname|collegename|
+----+-------+-------+--------+-----------+
|john|   85.0|    CSE|     CSE|        EGR|
|bill|   66.0|    ECE|     ECE|        EGR|
| lee|   75.0|  CIVIL|   CIVIL|        EGR|
|mary|   87.0|BIOLOGY| BIOLOGY|     NATSCI|
+----+-------+-------+--------+-----------+



In [13]:
res = studentsDF.groupBy(studentsDF['collegename']).agg({'hwgrade':'avg'})
res.show()

+-----------+-----------------+
|collegename|     avg(hwgrade)|
+-----------+-----------------+
|     NATSCI|             87.0|
|        EGR|75.33333333333333|
+-----------+-----------------+



### We can do all this with SQL also

In [14]:
# First, register the table for SQL so it will be recognized in the SQL statement
studentsDF.registerTempTable("studentsDF")

In [15]:
res = spark.sql("select collegename, AVG(hwgrade) from studentsDF group by studentsDF.collegename order by 2")
res.show()

+-----------+-----------------+
|collegename|     avg(hwgrade)|
+-----------+-----------------+
|        EGR|75.33333333333333|
|     NATSCI|             87.0|
+-----------+-----------------+



In [16]:
# If we use """ we can do it on multiple lines to make it easier to read
res = spark.sql("""
           select collegename, AVG(hwgrade) from studentsDF 
           group by studentsDF.collegename 
           order by 2
           """)
res.show()

+-----------+-----------------+
|collegename|     avg(hwgrade)|
+-----------+-----------------+
|        EGR|75.33333333333333|
|     NATSCI|             87.0|
+-----------+-----------------+



In [17]:
# We can do a more complex query
hwtotalDF.registerTempTable('hwtotalDF')
collegeDF.registerTempTable('collegeDF')
res = spark.sql("""
            select collegename, AVG(hwgrade) from hwtotalDF join collegeDF
            on hwtotalDF.dept == collegeDF.deptname 
            group by collegename
            order by 2
            """)
res.show()

+-----------+-----------------+
|collegename|     avg(hwgrade)|
+-----------+-----------------+
|        EGR|75.33333333333333|
|     NATSCI|             87.0|
+-----------+-----------------+



In [18]:
gradeDF.registerTempTable('gradeDF')
spark.sql("select * from gradeDF").show()

+----+---+---+---+
|name|hw1|hw2|hw3|
+----+---+---+---+
|john| 26| 29| 30|
|mary| 27| 25| 35|
|bill| 19| 22| 25|
| lee| 25| 25| 25|
+----+---+---+---+



In [19]:
gradeDF = spark.sql("select *, hw1+hw2+hw3 as hwtotoal from gradeDF")
gradeDF.show()

+----+---+---+---+--------+
|name|hw1|hw2|hw3|hwtotoal|
+----+---+---+---+--------+
|john| 26| 29| 30|    85.0|
|mary| 27| 25| 35|    87.0|
|bill| 19| 22| 25|    66.0|
| lee| 25| 25| 25|    75.0|
+----+---+---+---+--------+



In [22]:
gradeDF.registerTempTable('gradeDF')
spark.sql("select * from gradeDF").show()

+----+---+---+---+--------+
|name|hw1|hw2|hw3|hwtotoal|
+----+---+---+---+--------+
|john| 26| 29| 30|    85.0|
|mary| 27| 25| 35|    87.0|
|bill| 19| 22| 25|    66.0|
| lee| 25| 25| 25|    75.0|
+----+---+---+---+--------+

